In [124]:
import pandas as pd

In [125]:
bucket_name = "/data/"
file_key = "Cleaned_car_data.csv"

In [126]:
directory =  "D:/Books_and_notes/Wings1/T15_AWS/handson"+bucket_name+file_key
print(directory)
try:
    df = pd.read_csv(directory, header = 0)
    print(df.head())
except Exception as e:
    print("error while reading data",e)

D:/Books_and_notes/Wings1/T15_AWS/handson/data/Cleaned_car_data.csv
                        Car_name  year  selling_price  km_driven    fuel  \
0                Tata Indica DLS  2003          50000      70000  Diesel   
1           Maruti Swift 1.3 VXi  2008         130000     120000  Petrol   
2         Hyundai Santro Xing XP  2003          50000      80000  Petrol   
3      Mahindra Scorpio SLE BSIV  2011         400000     110000  Diesel   
4  Mahindra Bolero 2011-2019 SLE  2014         430000     120000  Diesel   

  seller_type transmission         Owner  
0  Individual       Manual   First Owner  
1  Individual       Manual  Second Owner  
2  Individual       Manual  Second Owner  
3  Individual       Manual  Second Owner  
4  Individual       Manual   First Owner  


In [127]:
try:
    current_year = 2024
    df['car_age'] = current_year - df['year']
    df.drop(['Car_name','year'],  axis = 1,inplace=True)
    print(df.head())
except Exception as e:
    print("error while droping columns",e)


   selling_price  km_driven    fuel seller_type transmission         Owner  \
0          50000      70000  Diesel  Individual       Manual   First Owner   
1         130000     120000  Petrol  Individual       Manual  Second Owner   
2          50000      80000  Petrol  Individual       Manual  Second Owner   
3         400000     110000  Diesel  Individual       Manual  Second Owner   
4         430000     120000  Diesel  Individual       Manual   First Owner   

   car_age  
0       21  
1       16  
2       21  
3       13  
4       10  


In [128]:
try:
    X = df.drop('selling_price', axis=1)
    y = df['selling_price']
    print(X.head(),y.head())
    print(X.info())
except Exception as e:
    print("error while definig features",e)

   km_driven    fuel seller_type transmission         Owner  car_age
0      70000  Diesel  Individual       Manual   First Owner       21
1     120000  Petrol  Individual       Manual  Second Owner       16
2      80000  Petrol  Individual       Manual  Second Owner       21
3     110000  Diesel  Individual       Manual  Second Owner       13
4     120000  Diesel  Individual       Manual   First Owner       10 0     50000
1    130000
2     50000
3    400000
4    430000
Name: selling_price, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4491 entries, 0 to 4490
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   km_driven     4491 non-null   int64 
 1   fuel          4491 non-null   object
 2   seller_type   4491 non-null   object
 3   transmission  4491 non-null   object
 4   Owner         4491 non-null   object
 5   car_age       4491 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 210.6+ K

In [129]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
import numpy as np

In [130]:
try:
    numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = X.select_dtypes(include=['object']).columns
    print(numerical_features, categorical_features)
except Exception as e:
        print("error while categrorising the data featuress",e)

Index(['km_driven', 'car_age'], dtype='object') Index(['fuel', 'seller_type', 'transmission', 'Owner'], dtype='object')


In [131]:
X[numerical_features] = X[numerical_features].apply(lambda x: np.log1p(x))

In [132]:
print(X)

      km_driven    fuel seller_type transmission         Owner   car_age
0     11.156265  Diesel  Individual       Manual   First Owner  3.091042
1     11.695255  Petrol  Individual       Manual  Second Owner  2.833213
2     11.289794  Petrol  Individual       Manual  Second Owner  3.091042
3     11.608245  Diesel  Individual       Manual  Second Owner  2.639057
4     11.695255  Diesel  Individual       Manual   First Owner  2.397895
...         ...     ...         ...          ...           ...       ...
4486  11.820418  Diesel  Individual       Manual   Third Owner  2.397895
4487  10.596660  Petrol  Individual       Manual   First Owner  2.079442
4488  11.695255  Petrol  Individual       Manual  Second Owner  2.484907
4489  11.407576  Petrol  Individual       Manual   First Owner  2.484907
4490  11.759793  Diesel      Dealer       Manual   First Owner  2.302585

[4491 rows x 6 columns]


In [133]:
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

In [134]:
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [135]:
preprocessor = ColumnTransformer(transformers=[('num',numeric_transformer,numerical_features),('cat',categorical_transformer,categorical_features)])

In [136]:
X_preprocessed = preprocessor.fit_transform(X)

In [137]:
X_preprocessed_df = pd.DataFrame(X_preprocessed,columns=np.concatenate((numerical_features,preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_features))))

In [138]:
print(X_preprocessed_df.head())

   km_driven   car_age  fuel_CNG  fuel_Diesel  fuel_LPG  fuel_Petrol  \
0  -0.434873  1.943677       0.0          1.0       0.0          0.0   
1   0.737149  1.016126       0.0          0.0       0.0          1.0   
2  -0.144516  1.943677       0.0          0.0       0.0          1.0   
3   0.547946  0.317641       0.0          1.0       0.0          0.0   
4   0.737149 -0.549950       0.0          1.0       0.0          0.0   

   seller_type_Dealer  seller_type_Individual  seller_type_Trustmark Dealer  \
0                 0.0                     1.0                           0.0   
1                 0.0                     1.0                           0.0   
2                 0.0                     1.0                           0.0   
3                 0.0                     1.0                           0.0   
4                 0.0                     1.0                           0.0   

   transmission_Automatic  transmission_Manual  Owner_First Owner  \
0                     0

In [139]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [142]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=8))])

In [143]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size =0.2,random_state=8)

In [144]:
param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [None, 10, 20, 30],
}

In [145]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='r2', n_jobs=1)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         Index(['km_driven', 'car_age'], dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         Index(['fuel', 'seller_type', 'transmission', 'Owner'], dtype='object'))])),
                                       ('regressor',
                                        RandomForestRegressor(random_state=8))]),
             n_jobs=1,
             param_grid={'regressor__max_depth': [None, 10, 20, 30],
                         'regressor__n_estimators': [100, 200, 300]},
             scoring='r2')

In [146]:
print("Best parameters found: ", grid_search.best_params_)
print("Best CV score: ", grid_search.best_score_)

Best parameters found:  {'regressor__max_depth': 10, 'regressor__n_estimators': 300}
Best CV score:  0.55747868766824


In [147]:
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['km_driven', 'car_age'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['fuel', 'seller_type', 'transmission', 'Owner'], dtype='object'))])),
                ('regressor',
                 RandomForestRegressor(max_depth=10, n_estimators=300,
                                       random_state=8))])

In [148]:
y_pred = best_model.predict(X_test)

In [149]:
def rmse(y_true, y_pred):
    return math.sqrt(mean_squared_error(y_true, y_pred))

In [152]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse_value = rmse(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [153]:
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse_value}")
print(f"R2 Score: {r2}")

Mean Absolute Error (MAE): 96696.42518307599
Mean Squared Error (MSE): 18172881440.098293
Root Mean Squared Error (RMSE): 134806.8300943921
R2 Score: 0.5166471109078845


In [154]:
print("Best hyperparameters:", grid_search.best_params_)

Best hyperparameters: {'regressor__max_depth': 10, 'regressor__n_estimators': 300}


In [160]:
from joblib import dump, load

In [161]:
dump(model, "D:/Books_and_notes/Wings1/T15_AWS/rfr.joblib")

['D:/Books_and_notes/Wings1/T15_AWS/rfr.joblib']

In [166]:
sklearn.pipeline??